In [1]:
class NPPGlobal:
    '''
    class NPPsurf_OCNPP_comp(runname,resultpath,savepath,mesh,matfileNPPsurf,first_year,last_year,
                 mapproj='rob',savefig=False, verbose=False, output=False, 
                            plotting=True, Taylor=True)
                 
    n_levels = 1: number of mesh levels used for FESOM surface mean
    
    self.NPPnfesom_interp contains 2D dataset of 1x1 interpolated nanophytoplankton NPP
    self.NPPdfesom_interp contains 2D dataset of 1x1 interpolated diatom NPP
    self.NPPtfesom_interp contains 2D dataset of 1x1 interpolated of total NPP
    self.unitfesom contains str of FESOM NPP unit
    self.OCNPP contains 2D dataset of 1x1 interpolated of remotely sensed NPP
    self.lon longitude
    self.lat latitude
    '''
    
    def __init__(self,resultpath,savepath,mesh,matfileNPPsurf,first_year,last_year,
                 mapproj='rob',runname='fesom',
                 savefig=False,output=False,plotting=True,verbose=False,Taylor=True):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.savefig = savefig
        self.matfileNPPsurf=matfileNPPsurf
        self.verbose = verbose
        self.taylor = Taylor
        self.output = output
        self.plotting = plotting
        self.Taylor = Taylor
        

        import matplotlib.pyplot as plt
        import matplotlib.colors as colors
        import numpy as np
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import scipy.io as spio
        import cartopy.feature as cfeature
        import pyfesom2 as pf    
        from Py_f2recom_toolbox import plt_Taylor_norm
        
        if self.mapproj == 'rob':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'pc':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'sp':
            box=[-180, 180, -90, -30]
        elif self.mapproj == 'np':
            box=[-180, 180, 60, 90]
            
        self.mapproj = pf.get_proj(self.mapproj)

        if(self.verbose):
            print('Processing {0}'.format(self.resultpath))
        
        # load OCCCI CHl.a data -------------------------------------------------------------------------------------
        matNPP = spio.loadmat(self.matfileNPPsurf, squeeze_me=True)
        
        lat         = np.arange(-90,90.-.1,1.)
        lon         = np.arange(-179.5,180.-.1,1.)
        latdic, londic = np.meshgrid(lat, lon)
        
        #annualchl   = np.log10(matChl['x'])
        npp = matNPP['NPP_CLIM']
        npp = np.nanmean(npp,axis=2) # For now, only take annual mean, seasonnal evaluation will be implemented later
        npp = npp.T
        
        OCNPPlabel = matfileNPPsurf[-1-12:-9]
        OCNPPunit = 'NPP [mg C m$^{-2}$ d$^{-1}$]'
        print(' !!! Satellite data for the 1998-2019 period only !!!')
        
        # load FESOM mesh -------------------------------------------------------------------------------------
        years = np.arange(self.fyear, self.lyear+1,1)
        
        lon_fesom = mesh.x2
        lat_fesom = mesh.y2        
        
        NPPnfesom = pf.get_data(self.resultpath, "NPPn", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)

        
        labelfesomNano = 'FESOM Nanophyto NPP {0}-{1}'.format(self.fyear,self.lyear)        

        
        NPPdfesom = pf.get_data(self.resultpath, "NPPd", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)
        
        labelfesomDia = 'FESOM Diatom NPP {0}-{1}'.format(self.fyear,self.lyear)
        
        from pathlib import Path
        cocco_path = Path(self.resultpath + '/NPPc.fesom.'+str(years[0])+'.nc') # assuming that coccos were used for the entire simulation if they were used in the first year of simulation
        phaeo_path = Path(self.resultpath + '/NPPp.fesom.'+str(years[0])+'.nc') # assuming that phaeo was used for the entire simulation if they were used in the first year of simulation
        
        
        if cocco_path.is_file():
            NPPcfesom = pf.get_data(self.resultpath, "NPPc", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)
            labelfesomCocco = 'FESOM Cocco NPP {0}-{1}'.format(self.fyear,self.lyear)
            
            if phaeo_path.is_file():
                NPPpfesom = pf.get_data(self.resultpath, "NPPp", years, mesh, 
                                   how="mean", compute=True, runid=self.runname, silent=True)
                labelfesomPhaeo = 'FESOM Phaeo NPP {0}-{1}'.format(self.fyear,self.lyear)
                print('4-phytoplankton model is used')
                
            else:
                print('3-phytoplankton model is used')
                
        else:
            print('2-phytoplankton model is used')
        
        
        # convert FESOM CHl.a data -------------------------------------------------------------------------------------
        # #########################
        # FESOM outputs the data in mg/m3 already, wrong units set in netcdf !?!?!
        # #########################
        if True:
            #C : 12.01 g/mol
            conv = 12.01
            NPPdfesom = NPPdfesom * conv # mmol/m2/day to mg/m2/day
            NPPnfesom = NPPnfesom * conv
            if cocco_path.is_file():
                NPPcfesom = NPPcfesom * conv
            if phaeo_path.is_file():
                NPPpfesom = NPPpfesom * conv
            unitfesom = 'NPP [mg C m$^{-2}$ d$^{-1}$]'
        else: 
            print('***\nFESOM data in not converted...\n***')
            unitfesom = 'NPP [mmol C m$^{-2}$ d$^{-1}$]'
         
        labelfesom = 'FESOM ({0}-{1})'.format(self.fyear,self.lyear)
        
        # interpolate FESOM CHl.a to regular -------------------------------------------------------------------------------------
        NPPn_interp = pf.fesom2regular(
                data = NPPnfesom,
                mesh = mesh,
                lons = londic, 
                lats = latdic)
        
        NPPd_interp = pf.fesom2regular(
                data = NPPdfesom,
                mesh = mesh,
                lons = londic, 
                lats = latdic)
        
        if cocco_path.is_file():
            NPPc_interp = pf.fesom2regular(
                data = NPPcfesom,
                mesh = mesh,
                lons = londic, 
                lats = latdic)

        if phaeo_path.is_file():
            NPPp_interp = pf.fesom2regular(
                data = NPPpfesom,
                mesh = mesh,
                lons = londic, 
                lats = latdic)
        
        # Nanophyto + Diatoms (+ Coccos + Phaeo): TOTAL NPP -------------------------------------------------------------------------------------
        
        NPPt_interp = NPPn_interp + NPPd_interp
        if cocco_path.is_file():
            NPPt_interp = NPPt_interp + NPPc_interp
        if phaeo_path.is_file():
            NPPt_interp = NPPt_interp + NPPp_interp
        


        # apply sea mask to OCCCI as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        OCNPP_ma = np.copy(npp)
        OCNPP_ma[~np.isfinite(NPPt_interp)] = np.nan
        
        # Derive regular grid area
        Aarea = pf.get_regulargrid_area(lon,lat)
        
        # Compute areal NPP
        NPP_OC_so = np.nansum(OCNPP_ma[:-1,lat<-50] * Aarea[:,lat[:-1]<-50])
        NPP_FE_so = np.nansum(NPPt_interp[:-1,lat<-50] * Aarea[:,lat[:-1]<-50])
        NPP_OC_glo = np.nansum(OCNPP_ma[:-1,:-1] * Aarea[:,:])
        NPP_FE_glo = np.nansum(NPPt_interp[:-1,:-1] * Aarea[:,:])
        print(' NPP SAT Southern Ocean [PgC / yr] = ', 365*NPP_OC_so/1e18)
        print(' NPP REcoM Southern Ocean [PgC / yr] = ', 365*NPP_FE_so/1e18)
        print(' NPP SAT Global [PgC / yr] = ', 365*NPP_OC_glo/1e18)
        print(' NPP REcoM Global [PgC / yr] = ', 365*NPP_FE_glo/1e18)
        
        # Convert to log10 scale
        OCNPP_ma_log10 = np.log10(OCNPP_ma)
        NPPn_log10     = np.log10(NPPn_interp)
        NPPd_log10     = np.log10(NPPd_interp)
        if cocco_path.is_file():
            NPPc_log10     = np.log10(NPPc_interp)
        if phaeo_path.is_file():
            NPPp_log10     = np.log10(NPPp_interp)
        NPPt_log10     = np.log10(NPPt_interp)
        
        # check CHl.a data -------------------------------------------------------------------------------------
        if(self.verbose):
            print('\nNPP\nOC nmin = {2:5.4f}, max = {3:5.4f}\nFESOM min = {0:5.4f}, max = {1:5.4f}'.format(
                    np.nanmin(NPPt_interp),np.nanmax(NPPt_interp),
                    np.nanmin(OCNPP_ma),np.nanmax(OCNPP_ma)))
        
        if (self.plotting):
            # plot each PP dataset -------------------------------------------------------------------------------------        
            levels = 1000*np.array([0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,
                               0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,
                               1,2,3]) #+,4,5,7?

            ticks = np.array([0,0.01,0.03,0.05,0.07,0.1,0.3,0.5,0.7,1,3])*1000

            def mygrid(m):
                #m.coastlines(resolution='110m', color='black', linewidth=1)
                m.add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='gray')
                #m.set_xlim(box[0], box[1])
                #m.set_ylim(box[2], box[3])


            # if phaeos and coccos are used ----------------------------------------------------------------------------------------
            
            if cocco_path.is_file() & phaeo_path.is_file():
           
                fig = plt.figure(figsize=(15,15), constrained_layout=False)
                axes = fig.subplot_mosaic(
                        """
                        AB
                        CD
                        EF
                        GG
                        """,
                        gridspec_kw={'hspace': 0.1, 'wspace': 0.1}, 
                        subplot_kw=dict(projection=self.mapproj))             

                # FESOM nanophyto
                m1 = axes['A']
                f1 = m1.pcolormesh(londic, latdic, NPPn_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                                   #vmin=1e-3,vmax=5e3)
                mygrid(m1)
                m1.set_extent(box, ccrs.PlateCarree())
                m1.set_title('FESOM-REcoM small phytoplankton', fontsize=16)


                # FESOM diatom
                m2 = axes['B']
                f2 = m2.pcolormesh(londic, latdic, NPPd_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m2)
                m2.set_extent(box, ccrs.PlateCarree())
                m2.set_title('FESOM-REcoM Diatom', fontsize=16)
                
                # FESOM coccolithophores
                m3 = axes['C']
                f3 = m3.pcolormesh(londic, latdic, NPPc_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m3)
                m3.set_extent(box, ccrs.PlateCarree())
                m3.set_title('FESOM-REcoM Coccolithophores', fontsize=16)
                
                # FESOM phaeocystis
                m4 = axes['D']
                f4 = m4.pcolormesh(londic, latdic, NPPp_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m4)
                m4.set_extent(box, ccrs.PlateCarree())
                m4.set_title('FESOM-REcoM Phaeocystis', fontsize=16)
                

                # OC-CCI
                m6 = axes['F']
                f6 = m6.pcolormesh(londic, latdic, OCNPP_ma, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                                   #vmin=1e-3,vmax=5e3)
                mygrid(m6)
                m6.set_extent(box, ccrs.PlateCarree())
                m6.set_title(OCNPPlabel, fontsize=16)
                


                # FESOM
                m5 = axes['E']
                f5 = m5.pcolormesh(londic, latdic, NPPt_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m5)
                m5.set_extent(box, ccrs.PlateCarree())
                m5.set_title('FESOM-REcoM Total', fontsize=16)

                cbar1_ax = fig.add_axes([0.92, 0.44, 0.02, 0.4])

                cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'vertical',
                                ticks = ticks,
                                fraction=0.1, pad=0.1,
                                extend = 'max') 
                cbar1.set_label(unitfesom, fontsize=14)
                cbar1.ax.tick_params(labelsize=14)



                # OC-CCI - FESOM
                levels_diff = np.arange(-600,620,20)
                m7 = axes['G']
                f7 = m7.pcolormesh(londic, latdic, NPPt_interp - OCNPP_ma, 
                                   transform = ccrs.PlateCarree(),
                                   cmap = 'RdBu_r',
                                   norm=colors.BoundaryNorm(boundaries=levels_diff, ncolors=256))
                #f3.set_clim([-2, 2])

                mygrid(m7)
                m7.set_extent(box, ccrs.PlateCarree())
                m7.set_title('FESOM-REcoM - '+matfileNPPsurf[-1-12:-9], fontsize=16)


                 # add one colorbar for difference plot below figure

                #fig.subplots_adjust(right=0.8)
                cbar2_ax = fig.add_axes([0.92, 0.14, 0.02, 0.2])

                cbar2 = fig.colorbar(f7,
                                cax = cbar2_ax, 
                                orientation = 'vertical',
                                extend = 'both',
                                #location ='bottom',
                                ticks = [-600,-400,-200,0,200,400,600]) 
                cbar2.ax.tick_params(labelsize=14)
                cbar2.set_label(unitfesom, fontsize=16)

                
                m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                            size=30, weight='bold')
                m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                            size=30, weight='bold')
                m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                            size=30, weight='bold')
                m4.text(-0.12, 1.05, 'D', transform=m4.transAxes,
                            size=30, weight='bold')
                m5.text(-0.12, 1.05, 'E', transform=m5.transAxes,
                            size=30, weight='bold')
                m6.text(-0.12, 1.05, 'F', transform=m6.transAxes,
                            size=30, weight='bold')
                m7.text(-0.12, 1.05, 'G', transform=m7.transAxes,
                            size=30, weight='bold')
                

                m1.set_extent(box, ccrs.PlateCarree())
                m2.set_extent(box, ccrs.PlateCarree())
                m3.set_extent(box, ccrs.PlateCarree())
                m4.set_extent(box, ccrs.PlateCarree())
                m5.set_extent(box, ccrs.PlateCarree())
                m6.set_extent(box, ccrs.PlateCarree())
                m7.set_extent(box, ccrs.PlateCarree())
            
            # if coccos are used ---------------------------------------------------------------------------------------------   
            
            elif cocco_path.is_file() and not phaeo_path.is_file():    
                fig = plt.figure(figsize=(15,15), constrained_layout=False)
                axes = fig.subplot_mosaic(
                        """
                        ABC
                        DEF
                        GGG
                        """,
                        gridspec_kw={'hspace': 0.1, 'wspace': 0.1}, 
                        subplot_kw=dict(projection=self.mapproj))             

                # FESOM nanophyto
                m1 = axes['A']
                f1 = m1.pcolormesh(londic, latdic, NPPn_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                                   #vmin=1e-3,vmax=5e3)
                mygrid(m1)
                m1.set_extent(box, ccrs.PlateCarree())
                m1.set_title('FESOM-REcoM small phytoplankton', fontsize=16)


                # FESOM diatom
                m2 = axes['B']
                f2 = m2.pcolormesh(londic, latdic, NPPd_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m2)
                m2.set_extent(box, ccrs.PlateCarree())
                m2.set_title('FESOM-REcoM Diatom', fontsize=16)
                
                # FESOM coccolithophores
                m3 = axes['C']
                f3 = m3.pcolormesh(londic, latdic, NPPc_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m3)
                m3.set_extent(box, ccrs.PlateCarree())
                m3.set_title('FESOM-REcoM Coccolithophores', fontsize=16)
                
                
                

                # OC-CCI
                m5 = axes['F']
                f5 = m5.pcolormesh(londic, latdic, OCNPP_ma, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                                   #vmin=1e-3,vmax=5e3)
                mygrid(m5)
                m5.set_extent(box, ccrs.PlateCarree())
                m5.set_title(OCNPPlabel, fontsize=16)
                
                # Placeholder
                m7 = axes['E']
                axes['E'].remove()

                # FESOM
                m4 = axes['D']
                f4 = m4.pcolormesh(londic, latdic, NPPt_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m4)
                m4.set_extent(box, ccrs.PlateCarree())
                m4.set_title('FESOM-REcoM Total', fontsize=16)

                cbar1_ax = fig.add_axes([0.92, 0.44, 0.02, 0.4])

                cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'vertical',
                                ticks = ticks,
                                fraction=0.1, pad=0.1,
                                extend = 'max') 
                cbar1.set_label(unitfesom, fontsize=14)
                cbar1.ax.tick_params(labelsize=14)



                # OC-CCI - FESOM
                levels_diff = np.arange(-600,620,20)
                m6 = axes['G']
                f6 = m6.pcolormesh(londic, latdic, NPPt_interp - OCNPP_ma, 
                                   transform = ccrs.PlateCarree(),
                                   cmap = 'RdBu_r',
                                   norm=colors.BoundaryNorm(boundaries=levels_diff, ncolors=256))
                #f3.set_clim([-2, 2])

                mygrid(m6)
                m6.set_extent(box, ccrs.PlateCarree())
                m6.set_title('FESOM-REcoM - '+matfileNPPsurf[-1-12:-9], fontsize=16)


                # add one colorbar for difference plot below figure

                #fig.subplots_adjust(right=0.8)
                cbar2_ax = fig.add_axes([0.92, 0.14, 0.02, 0.2])

                cbar2 = fig.colorbar(f6,
                                cax = cbar2_ax, 
                                orientation = 'vertical',
                                extend = 'both',
                                #location ='bottom',
                                ticks = [-600,-400,-200,0,200,400,600]) 
                cbar2.ax.tick_params(labelsize=14)
                cbar2.set_label(unitfesom, fontsize=16)

                
                m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                            size=30, weight='bold')
                m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                            size=30, weight='bold')
                m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                            size=30, weight='bold')
                m4.text(-0.12, 1.05, 'D', transform=m4.transAxes,
                            size=30, weight='bold')
                m5.text(-0.12, 1.05, 'E', transform=m5.transAxes,
                            size=30, weight='bold')
                m6.text(-0.12, 1.05, 'F', transform=m6.transAxes,
                            size=30, weight='bold')
                m7.text(-0.12, 1.05, '', transform=m7.transAxes,
                            size=30, weight='bold')

                m1.set_extent(box, ccrs.PlateCarree())
                m2.set_extent(box, ccrs.PlateCarree())
                m3.set_extent(box, ccrs.PlateCarree())
                m4.set_extent(box, ccrs.PlateCarree())
                m5.set_extent(box, ccrs.PlateCarree())
                m6.set_extent(box, ccrs.PlateCarree())
                
                
            # if coccos are not used ------------------------------------------------------------------------------------    
            
            else:
                fig = plt.figure(figsize=(15,15), constrained_layout=False)
                axes = fig.subplot_mosaic(
                        """
                        AB
                        CD
                        EE
                        """,
                        gridspec_kw={'hspace': 0.1, 'wspace': 0.1}, 
                        subplot_kw=dict(projection=self.mapproj))             

                # FESOM nanophyto
                m1 = axes['A']
                f1 = m1.pcolormesh(londic, latdic, NPPn_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                                   #vmin=1e-3,vmax=5e3)
                mygrid(m1)
                m1.set_extent(box, ccrs.PlateCarree())
                m1.set_title('FESOM-REcoM small phytoplankton', fontsize=16)


                # FESOM diatom
                m2 = axes['B']
                f2 = m2.pcolormesh(londic, latdic, NPPd_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m2)
                m2.set_extent(box, ccrs.PlateCarree())
                m2.set_title('FESOM-REcoM Diatom', fontsize=16)

                # OC-CCI
                m4 = axes['D']
                f4 = m4.pcolormesh(londic, latdic, OCNPP_ma, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                                   #vmin=1e-3,vmax=5e3)
                mygrid(m4)
                m4.set_extent(box, ccrs.PlateCarree())
                m4.set_title(OCNPPlabel, fontsize=16)


                # FESOM
                m3 = axes['C']
                f3 = m3.pcolormesh(londic, latdic, NPPt_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m3)
                m3.set_extent(box, ccrs.PlateCarree())
                m3.set_title('FESOM-REcoM Total', fontsize=16)

                cbar1_ax = fig.add_axes([0.92, 0.44, 0.02, 0.4])

                cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'vertical',
                                ticks = ticks,
                                fraction=0.1, pad=0.1,
                                extend = 'max') 
                cbar1.set_label(unitfesom, fontsize=14)
                cbar1.ax.tick_params(labelsize=14)



                # OC-CCI - FESOM
                levels_diff = np.arange(-600,620,20)
                m5 = axes['E']
                f5 = m5.pcolormesh(londic, latdic, NPPt_interp - OCNPP_ma, 
                                   transform = ccrs.PlateCarree(),
                                   cmap = 'RdBu_r',
                                   norm=colors.BoundaryNorm(boundaries=levels_diff, ncolors=256))
                #f3.set_clim([-2, 2])

                mygrid(m5)
                m5.set_extent(box, ccrs.PlateCarree())
                m5.set_title('FESOM-REcoM - '+matfileNPPsurf[-1-12:-9], fontsize=16)

                # add one colorbar for difference plot below figure

                #fig.subplots_adjust(right=0.8)
                cbar2_ax = fig.add_axes([0.92, 0.14, 0.02, 0.2])

                cbar2 = fig.colorbar(f5,
                                cax = cbar2_ax, 
                                orientation = 'vertical',
                                extend = 'both',
                                #location ='bottom',
                                ticks = [-600,-400,-200,0,200,400,600]) 
                cbar2.ax.tick_params(labelsize=14)
                cbar2.set_label(unitfesom, fontsize=16)
                
                
                m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                            size=30, weight='bold')
                m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                            size=30, weight='bold')
                m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                            size=30, weight='bold')
                m4.text(-0.12, 1.05, 'D', transform=m4.transAxes,
                            size=30, weight='bold')
                m5.text(-0.12, 1.05, 'E', transform=m5.transAxes,
                            size=30, weight='bold')

                m1.set_extent(box, ccrs.PlateCarree())
                m2.set_extent(box, ccrs.PlateCarree())
                m3.set_extent(box, ccrs.PlateCarree())
                m4.set_extent(box, ccrs.PlateCarree())
                m5.set_extent(box, ccrs.PlateCarree())

            
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'OCNPP_'+matfileNPPsurf[-1-12:-9]+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'OCNPP_'+matfileNPPsurf[-1-12:-9]+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                        bbox_inches='tight')
            plt.show(block=False)  

        if(self.Taylor):
            # statistics  -------------------------------------------------------------------------------------            
            # preparation of datasets
            if np.isnan(np.min(OCNPP_ma_log10)): print('WARNING: OCNPP field contains NaNs')
            if np.isnan(np.min(NPPt_log10)): print('WARNING: FESOM field contains NaNs')

            # get statistics only from valid OCCCI gridpoints 
            ind_stat = np.where(np.isfinite(OCNPP_ma_log10))

            title = 'log10 surface NPP'
            print('\nStatistics for '+title)
            plt_Taylor_norm(OCNPP_ma_log10[ind_stat],NPPt_log10[ind_stat],
                                        mask=True,title=title)
            
            
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):                
                plt.savefig(self.savepath+self.runname+'_'+'OCNPP_Taylor_'+matfileNPPsurf[-1-12:-9]+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'OCNPP_Taylor_'+matfileNPPsurf[-1-12:-9]+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                            bbox_inches='tight')
            plt.show(block=False) 
        
        # store interpolated datasets ----------------------------------------------------------------------------------
        if output:
            self.NPPd_interp = NPPd_interp
            self.NPPn_interp = NPPn_interp
            if cocco_path.is_file():
                self.NPPc_interp = NPPc_interp
            if phaeo_path.is_file():
                self.NPPp_interp = NPPp_interp
            self.NPPt_interp = NPPt_interp
            self.unit = unitfesom
            self.NPPt_OC = OCNPP_ma